# XLSX Writer

In [1]:
import pandas as pd

# Create a test df
df = pd.DataFrame({'station': ['BKK','BKK','BKK', 'BKK','BKK','BKK', 'CNX','CNX','KKN', 'KKN', 'KKN', 'KKN', 'KKN', 'KKN'],
                   'Time': ['7/1/2020  12:00:00', '7/1/2020  12:00:00', '7/1/2020  12:00:00', '7/1/2020  18:00:00', '7/1/2020  18:00:00', '7/1/2020  18:00:00','7/1/2020  12:00:00','7/1/2020  12:00:00', '7/1/2020  12:00:00', '7/1/2020  00:00:00', '7/1/2020  00:00:00', '7/1/2020  06:00:00', '7/1/2020  06:00:00', '7/1/2020  12:00:00'],
                   'Predicted': ['7/1/2020  6:00:00', '7/1/2020  12:00:00', '7/1/2020  18:00:00', '7/1/2020  6:00:00', '7/1/2020  12:00:00', '7/1/2020  18:00:00', '7/2/2020  0:00:00', '7/2/2020  6:00:00', '7/2/2020  12:00:00', '7/2/2020  12:00:00', '7/2/2020  18:00:00', '7/3/2020  00:00:00' ,'7/3/2020  06:00:00' ,'7/3/2020  12:00:00'],
                   'PM2.5': [4, 18, 79, 40, 18, 79, 40,69, 16, 18, 79, 40,69, 16]})
df.reset_index()

,index,station,Time,Predicted,PM2.5
0,0,BKK,7/1/2020 12:00:00,7/1/2020 6:00:00,4
1,1,BKK,7/1/2020 12:00:00,7/1/2020 12:00:00,18
2,2,BKK,7/1/2020 12:00:00,7/1/2020 18:00:00,79
3,3,BKK,7/1/2020 18:00:00,7/1/2020 6:00:00,40
4,4,BKK,7/1/2020 18:00:00,7/1/2020 12:00:00,18
5,5,BKK,7/1/2020 18:00:00,7/1/2020 18:00:00,79
6,6,CNX,7/1/2020 12:00:00,7/2/2020 0:00:00,40
7,7,CNX,7/1/2020 12:00:00,7/2/2020 6:00:00,69
8,8,KKN,7/1/2020 12:00:00,7/2/2020 12:00:00,16
9,9,KKN,7/1/2020 00:00:00,7/2/2020 12:00:00,18


In [5]:
fileName = 'test.xlsx'

# Create a list where we 'll capture the cells that appear for 1st time,
# add the 1st row and we start checking from 2nd row until end of df
startCells_station = [1]
startCells_time = [1]

for row in range(2,len(df)+1):
    if df.loc[row-1,'station'] != df.loc[row-2,'station']:
        startCells_station.append(row)
        startCells_time.append(row)
    elif df.loc[row-1,'Time'] != df.loc[row-2,'Time']:
        startCells_time.append(row)

writer = pd.ExcelWriter(fileName, engine='xlsxwriter')
df.to_excel(writer, sheet_name='sheet1', index=True)
workbook = writer.book
worksheet = writer.sheets['sheet1']
cell_format = workbook.add_format()
cell_format.set_align('center')
cell_format.set_align('top')

lastRow = len(df)

# merge the Time column
for row in startCells_time:
    try:
        endRow_time = startCells_time[startCells_time.index(row)+1]-1
        if row == endRow_time:
            worksheet.write(row, 1, df.loc[row-1,'station'], cell_format)
        else:
            # merge Time
            worksheet.merge_range(row, 2, endRow_time, 2, df.loc[row-1,'Time'], cell_format)
    except IndexError:
        if row == lastRow:
            worksheet.write(row, 1, df.loc[row-1,'station'], cell_format)
        else:
            worksheet.merge_range(row, 2, lastRow, 2, df.loc[row-1,'Time'], cell_format)

# merge the station column
for row in startCells_station:
    try:
        endRow_station = startCells_station[startCells_station.index(row)+1]-1
        if row == endRow_time:
            continue
        else:
            # merge station
            worksheet.merge_range(row, 1, endRow_station, 1, df.loc[row-1,'station'], cell_format)
    except IndexError:
        if row == lastRow:
            continue
        else:
            worksheet.merge_range(row, 1, lastRow, 1, df.loc[row-1,'station'], cell_format)

writer.save()